In [1]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import spacy
from geotext import GeoText
import numpy as np
from multiprocessing import Pool
from collections import Counter
from datetime import datetime, timedelta

In [ ]:
print('hello')

In [2]:
df=pd.read_csv('Eluvio_DS_Challenge.csv')

In [3]:
df.head()

,time_created,date_created,up_votes,down_votes,title,over_18,author,category
0,1201232046,2008-01-25,3,0,Scores killed in Pakistan clashes,False,polar,worldnews
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission,False,polar,worldnews
2,1201232523,2008-01-25,3,0,US presses Egypt on Gaza border,False,polar,worldnews
3,1201233290,2008-01-25,1,0,Jump-start economy: Give health care to all,False,fadi420,worldnews
4,1201274720,2008-01-25,4,0,Council of Europe bashes EU&UN terror blacklist,False,mhermans,worldnews


In [ ]:
df['time_created']=pd.to_datetime(df['time_created'],unit='s')
df['date_created'] = pd.to_datetime(df['date_created'])

In [ ]:
df.info()

In [ ]:
for column in ['up_votes','down_votes','author','over_18','category']:
    uniques = df[column].unique()
    print('{0:20s} {1:5d}\t'.format(column, len(uniques)), uniques[:5])

In [ ]:
print(df['over_18'].value_counts())
sns.countplot(df['over_18']).set_title("LaLaLa")

In [ ]:
print(df['up_votes'].describe())
f, axes = plt.subplots(1, 2,figsize=(15, 4))
(df['up_votes'].value_counts().sort_index().cumsum()/len(df['up_votes'])).plot(title='cdf',ax=axes[0])
sns.distplot(df[df['up_votes']<100]['up_votes'],ax=axes[1])

In [ ]:
del df['down_votes'], df['category']

In [ ]:
df['have_pic_video']=df['title'].str.contains('\[video\]|\[pic\]')
df['have_pic_video'].value_counts()

In [ ]:
import scipy
scipy.stats.ttest_ind(df[df.have_pic_video==False]['up_votes'],df[df.have_pic_video==True]['up_votes'],equal_var=False)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 6))
num = 15
sns.countplot(y=df['author'],order = df['author'].value_counts().iloc[:num].index, ax=ax[0])

g=pd.DataFrame(df.groupby('author')['up_votes'].sum().sort_values(ascending=False))
g['vote_sum']=g['up_votes']
sns.barplot(y=g.index[:num],x=g['vote_sum'].iloc[:num],ax=ax[1])

In [ ]:
df['title_length']=df['title'].apply(lambda x: len(x.split()))
f, ax = plt.subplots(1, 2, figsize=(15, 4))
sns.distplot(df['title_length'],ax=ax[0])
g=df.groupby('title_length')['up_votes'].mean()
sns.scatterplot(x=g.index,y=g,ax=ax[1])

In [ ]:
def get_time_feature(df):
    df['day_of_week']=df['date_created'].dt.dayofweek
    df['day_of_week_name']=df['date_created'].dt.day_name()
    df['year'] = df['date_created'].dt.year
    df['month'] = df['date_created'].dt.month
    df['day'] = df['date_created'].dt.day
    df['weekend']=(df['date_created'].dt.dayofweek).apply(lambda x: 1 if x>4 else 0)
    df['day_time_hour']=df['time_created'].dt.hour+df['time_created'].dt.minute/60+df['time_created'].dt.second/3600
    df['pm']=df['day_time_hour'].apply(lambda x: 1 if x>12 else 0)
    return df

In [ ]:
df=get_time_feature(df)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 4))
g=df.groupby('year')['up_votes'].agg(['size', 'mean'])
sns.barplot(x=g.index,y=g['size'],ax=ax[0]).set_title('Num of posts vs Year')
sns.barplot(x=g.index,y=g['mean'],ax=ax[1]).set_title('Avg upvotes vs Year')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 4))
g=df.groupby('day_of_week_name')['up_votes'].agg(['size', 'mean'])
sns.barplot(x=g.index,y=g['size'],ax=ax[0]).set_title('Num of posts vs Year')
sns.barplot(x=g.index,y=g['mean'],ax=ax[1]).set_title('Avg upvotes vs Year')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 4))
g=df.groupby('day_time_hour')['up_votes'].agg(['size', 'mean'])
sns.scatterplot(x=g.index, y=g['size'],ax=ax[0])
sns.scatterplot(x=g.index, y=g['mean'], ax=ax[1])

In [ ]:
df['title']=df['title'].str.replace(r'\s+',' ')
df['title']=df['title'].str.replace('\[video\]|\[pic\]','')
df['title']=df['title'].str.replace(r'\.','')
df['title']=df['title'].str.replace(r'[^a-zA-Z\s]',' ')

In [ ]:
df['country']=df['title'].apply(lambda x: GeoText(x).countries)

In [ ]:
df['country']=df['country'].apply(lambda x: x if x else np.nan)

In [ ]:
t=df[['country','year']].dropna()
t['country']=t['country'].apply(lambda x: x[0])
fig = plt.figure(figsize=(10,10))
num = 15
sns.countplot(y=t['country'],order = t['country'].value_counts().iloc[:num].index, hue=t['year'])

In [ ]:
def have_trend_word(x):
    end_date=x['date_created']
    start_date=x['date_created']-timedelta(days=8)
    g=df[(df.date_created>=start_date)&(df.date_created<end_date)]
    g=g[g.up_votes>5].dropna()['country']
    if len(g)==0:
        return 0
    trend_word=set(np.hstack(g.tolist()))
    count=0
    for i in x['title'].split():
        if i in trend_word:
            count=count+1
    return count

In [ ]:
count_list=[]
for index, row in df.iterrows():
    count_list.append(have_trend_word(row))
    print(index)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def remove_stopword_and_lemm(x):
    stop_words = set(stopwords.words('english'))
    new_word=[word for word in x.split() if word not in stop_words]
    lemma=WordNetLemmatizer() 
    clean_word=[lemma.lemmatize(w,'v') for w in new_word]
    return " ".join(clean_word)

def tokenize_word_count(x):
    x=x.str.replace(r'\st\s',"'t ")
    x=x.str.replace(r'\ss\s',"'s ")
    x=x.str.replace(r'\bUK\b','uk')
    x=x.str.replace(r'\bEU\b','eu')
    x=x.str.replace(r'\b([A-Z])[a-z]+\b', lambda m: m.group(0).lower())
    x=x.apply(remove_stopword_and_lemm)
    return x

In [ ]:
s=tokenize_word_count(df['title'])

In [ ]:
ss=' '.join(s).split()

In [ ]:
from wordcloud import WordCloud

In [ ]:
wordcloud = WordCloud(width=1600, height=800,background_color='white').generate_from_frequencies(Counter(ss))
fig = plt.figure(figsize=(12,8), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
# Helper function
vectorizer = CountVectorizer(lowercase=False)
count_data = vectorizer.fit_transform(s.sample(n=50000, random_state=1))
def print_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 10
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)
#Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, vectorizer, number_words)

In [ ]:
print("Topics found via LDA:")
print_topics(lda, vectorizer, number_words)

In [ ]:
df['title']=pd.concat(l)

In [ ]:
df['title']=df['title'].str.replace('\.','')

In [ ]:
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',max_features=2000)
X = vectorizer.fit_transform(X_train)


In [ ]:
import re

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [ ]:
def get_doc_vec(x):
    vec=[]
    word_list=x.split()
    for word in word_list:
        if word in wv.vocab:
            vec.append(wv[word])
    if vec:
        return np.mean(vec,axis=0)
    else:
        return np.nan
def f1(x):
    return x.apply(get_doc_vec)

In [ ]:
len(count_list)

In [ ]:
len(df)

In [ ]:
p = Pool(8)
l=p.map(f1, np.array_split(df['title'], 8))

In [ ]:
df['docvec']=pd.concat(l)

In [ ]:
df=df.dropna()

In [ ]:
df['label']=0

In [ ]:
df.loc[(df.up_votes>5),'label']=1

In [ ]:
df['label'].value_counts()

In [ ]:
len(df)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df, df['label'],test_size=0.3,random_state=101)

In [ ]:
X_train,X_val=author_encoding(X_train,X_val)

In [ ]:
X_train_feature=np.concatenate((np.array(X_train['docvec'].tolist()), 
                                get_categorical_feature(X_train),
                                X_train['title_length'].values.reshape(-1,1),
                               X_train['author_total_posts'].values.reshape(-1,1),
                               X_train['have_trend_word'].values.reshape(-1,1)), axis=1)
X_val_feature=np.concatenate((np.array(X_val['docvec'].tolist()), 
                              get_categorical_feature(X_val),
                              X_val['title_length'].values.reshape(-1,1),
                             X_val['author_total_posts'].values.reshape(-1,1),
                             X_val['have_trend_word'].values.reshape(-1,1)), axis=1)

In [ ]:
X_train_feature.shape

In [ ]:
get_categorical_feature(X_train).shape

In [ ]:
X_train.shape

In [ ]:
from keras.utils import to_categorical

In [ ]:
y=to_categorical(df['label'],num_classes=3)

In [ ]:
y_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train_feature[:,:300].shape

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train_feature, y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df.columns

In [ ]:
df.loc[df.up_votes>5,'label']=1

In [ ]:
df['label'].value_counts()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train_feature, y_train)

In [ ]:
accuracy_score(y_val, clf.predict(X_val_feature))

In [ ]:
clf.predict(X_train)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(150, input_dim=311, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_feature, y_train, epochs=150, batch_size=64)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_feature, y_train)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
from sklearn.manifold import TSNE
from adjustText import adjust_text
tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)
tsne_df = tsne.fit_transform(np.vstack(df['docvec'].iloc[0:400].to_list())) # Use only 400 rows to shorten processing time


In [ ]:
tsne_df[0,1]

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
sns.scatterplot(tsne_df[:, 0], tsne_df[:, 1], alpha = 0.5)

texts = []
titles_to_plot = list(np.arange(0, 400, 50)) # plots every 40th title in first 400 titles

# Append words to list
for title in titles_to_plot:
    texts.append(plt.text(tsne_df[title, 0], tsne_df[title, 1], df['title'].iloc[title], fontsize = 14))
    
# Plot text using adjust_text (because overlapping text is hard to read)
adjust_text(texts, force_points = 0.4, force_text = 0.4, 
            expand_points = (2,1), expand_text = (1,2),
            arrowprops = dict(arrowstyle = "-", color = 'black', lw = 0.5))

In [ ]:
df

In [ ]:
def get_categorical_feature(x):
    x=pd.concat([pd.get_dummies(x['pm']),pd.get_dummies(x['weekend']),
               pd.get_dummies(x['over_18']),pd.get_dummies(x['have_pic_video'])],axis=1)
    return x.values

In [ ]:
def author_encoding(train,test):
    post_dict=train.groupby('author').size().to_dict()
    train['author_total_posts']=train['author'].map(post_dict)
    test['author_total_posts']=test['author'].map(post_dict).fillna(1)
    return train,test

In [ ]:
get_categorical_feature(df).shape

In [ ]:
np.array(df['title_length'].tolist()).reshape(-1,1).shape

In [ ]:
df['author_total_posts'] = df['author'].groupby(df['author']).transform('count')

In [ ]:
df.columns

In [ ]:
X_val['author'].map(X_train.groupby('author').size().to_dict()).fillna(1)

In [ ]:
len(X_val)

In [ ]:
df['author_total_posts']=1

In [ ]:
df['author'].map(df.groupby('author').size().to_dict())